In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os
import sqlite3


conn = sqlite3.connect('./WAR_RP9_InProcess/trial-db/Harvest.db')
c = conn.cursor()





In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [ ]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WAR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WAR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

big_df['Date'] = pd.to_datetime(big_df['Date'])

harvest_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

harvest_df = harvest_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\WAR_RP6
WAR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP7
WAR_RP7_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP8
WAR_RP8_Compiled.xlsm
P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP2
WAR_RP2_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP3
WAR_RP3_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP4
WAR_RP4_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
harvest_df.loc[harvest_df['TicketID'] == 'WAR18051XXXX', 'TicketID'] = 'WAR180518XXXX' # Found some mislabeled tickets

In [ ]:
# Time for the mill ticket normalization
# Normalize the Weeks column.
# Normalize the source column
# Normalize the Mill column
# Normalize the Product column

In [ ]:
len(harvest_df['Mill'].unique().tolist())

In [ ]:
len(harvest_df['Mill'].str.lower().unique().tolist())

In [ ]:
mill_list = harvest_df['Mill'].unique().tolist()
mill_dict = {}
mill_dict['mill'] = mill_list
mill_df = pd.DataFrame(mill_dict)
mill_df['mill_uid'] = mill_df.index
mill_mapper = dict(zip(mill_df['mill'], mill_df['mill_uid']))
harvest_df['mill'] = harvest_df['Mill'].map(mill_mapper)
harvest_df.drop(columns=['Mill'], inplace=True)

In [ ]:
harvest_df['Week'] = harvest_df['Week'].astype(str)
harvest_df.loc[harvest_df['Week'] == 'Week3b', 'Week'] = "3"
def weeker(week):
    if "Week" in week:
        x = week.split("k")[1]
        x = int(x)
        return x
    else:
        return int(week)
    
harvest_df['Week_trial'] = harvest_df['Week'].apply(lambda x: weeker(x))

week_list = harvest_df['Week_trial'].unique().tolist()
week_dict = {}
week_dict['week'] = week_list
week_df = pd.DataFrame(week_dict)
week_df['week_uid'] = week_df.index
week_mapper = dict(zip(week_df['week'], week_df['week_uid']))
harvest_df['week'] = harvest_df['Week_trial'].map(week_mapper)

In [ ]:
product_list = harvest_df['Product'].unique().tolist()
product_dict = {}
product_dict['product'] = product_list
product_df = pd.DataFrame(product_dict)
product_df['product_uid'] = product_df.index
product_mapper = dict(zip(product_df['product'], product_df['product_uid']))
harvest_df['product'] = harvest_df['Product'].map(product_mapper)

In [ ]:
harvest_df

In [ ]:
# get ticketid from sql and see how that shakes out
ticketid_df = pd.read_sql('SELECT * FROM ticketid',conn)
ticket_id_mapper = dict(zip(ticketid_df['ticketid'], ticketid_df['id']))
harvest_df['ticketid'] = harvest_df['TicketID'].map(ticket_id_mapper)
harvest_df['ticketid'] = harvest_df['ticketid'].astype(int)

In [ ]:
harvest_df.drop(columns=['Product', 'TicketID', 'Week'],inplace=True)
harvest_df.rename(columns={'Tons (U.S)': 'total_tons'})

In [ ]:
harvest_df.columns = [x.lower() for x in harvest_df.columns]
harvest_df.columns = [x.replace(' ', '_') for x in harvest_df.columns]

In [ ]:
harvest_df[harvest_df['reporting_period'] == 2]